In [1]:
import gensim
import keras
import numpy as np
from CMVscraper_script_delta2 import *
from detect_and_score import *

c:\users\paul\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


Load Google's word2vec model

In [2]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format ('models/GoogleNews-vectors-negative300.bin', binary=True)  

In [3]:
#word2vec.similarity("cat", "dog")

Load in our own Keras LSTM models

In [4]:
detect_model = keras.models.load_model("models/bilstm_cos.h5")

In [5]:
#model2 = keras.models.load_model("models/bilstm_evidence.h5")

In [6]:
score_model = keras.models.load_model("models/argument_quality.h5")

In [7]:
#OPname, OPtxt, CommentText, commentName, URL, TOPIC, deltalink, deltabot = search("astrology")
OPname, OPtxt, CommentText, commentName, URL, TOPIC, deltalink, deltabot = urlSearch("0")

In [8]:
topic = TOPIC.split(":")[1]
topic

' People that use the terms "soy boy" or "cuck" are highly insecure about their own masculinity.'

In [9]:
OP_post = OPtxt
OP_post

"Whenever I hear those terms I can't stop myself from imagining that person as a highly insecure person that tries to prove to themselves how masculine they are by making fun of people that remind themselves of their own self-perceived flaws.Especially because the only people that are seemingly using those terms are incels, MGTOWs, red pillers and others who's whole world view is based on insecure delusions. And because I have never met a confident person in my life that would use such terms, but lots of complete losers that feel that it gives them some kind of power.CMV: Guys that are using those terms unironically as a way to make fun of men that aren't hypermasculine are insecure about their own masculinity. "

In [10]:
OP_arguments = detect_and_score(detect_model, score_model, OP_post, topic, word2vec)
comment_arguments = detect_and_score(detect_model, score_model, comment_post, topic, word2vec)
delta_arguments = detect_and_score(detect_model, score_model, delta_post, topic, word2vec)

In [11]:
arguments

[(array([0.49154076], dtype=float32),
  "Whenever I hear those terms I can't stop myself from imagining that person as a highly insecure person that tries to prove to themselves how masculine they are by making fun of people that remind themselves of their own self-perceived flaws"),
 (array([0.64216805], dtype=float32),
  "Especially because the only people that are seemingly using those terms are incels, MGTOWs, red pillers and others who's whole world view is based on insecure delusions"),
 (array([0.44514513], dtype=float32),
  "CMV: Guys that are using those terms unironically as a way to make fun of men that aren't hypermasculine are insecure about their own masculinity")]

In [12]:
calculate_score(arguments)

0.52628464